In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys, os, gc

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *

In [3]:
# import aggregated trips and reset index
trips_aggregated_df = pd.read_parquet(TRIPS_GROUPED_SPATIO_TEMPORAL_PATH)
trips_aggregated_df.head(2)

,datetime_start_floored,start_hex_id,end_hex_id,demand,h3_res,time_interval_length
index,,,,,,
0,2019-01-20,871f1a164ffffff,871f1a164ffffff,1,7,1
1,2019-01-20,871f1a164ffffff,871f1a8c8ffffff,2,7,1


In [4]:
trips_aggregated_df = trips_aggregated_df[
    trips_aggregated_df["h3_res"].apply(lambda x: x in CALC_H3_RESOLUTIONS)
]
print(trips_aggregated_df.h3_res.unique())
print(trips_aggregated_df.index.size)


[7 8 9]
4253147


In [5]:
# add time features
trips_aggregated_df['hour'] = trips_aggregated_df.datetime_start_floored.dt.hour
trips_aggregated_df['weekday'] = trips_aggregated_df.datetime_start_floored.dt.weekday
trips_aggregated_df['month'] = trips_aggregated_df.datetime_start_floored.dt.month
trips_aggregated_df.head(2)

,datetime_start_floored,start_hex_id,end_hex_id,demand,h3_res,time_interval_length,hour,weekday,month
index,,,,,,,,,
0,2019-01-20,871f1a164ffffff,871f1a164ffffff,1,7,1,0,6,1
1,2019-01-20,871f1a164ffffff,871f1a8c8ffffff,2,7,1,0,6,1


In [6]:
hour_dummies = pd.get_dummies(trips_aggregated_df['hour']).add_prefix('hour_')
weekday_dummies = pd.get_dummies(trips_aggregated_df['weekday']).add_prefix('weekday_')
month_dummies = pd.get_dummies(trips_aggregated_df['month']).add_prefix('month_')

trips_aggregated_df.drop(['hour', 'weekday', 'month'], axis=1, inplace=True)
trips_aggregated_df = pd.concat([trips_aggregated_df, hour_dummies, weekday_dummies, month_dummies], axis=1)
trips_aggregated_df.head(2)

,datetime_start_floored,start_hex_id,end_hex_id,demand,h3_res,time_interval_length,hour_0,hour_1,hour_2,hour_3,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
index,,,,,,,,,,,,,,,,,,,,,
0,2019-01-20,871f1a164ffffff,871f1a164ffffff,1,7,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-01-20,871f1a164ffffff,871f1a8c8ffffff,2,7,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
del hour_dummies, weekday_dummies, month_dummies
gc.collect()

20

In [8]:
weather_df = pd.read_parquet(WEATHER_AGGR_TEMPORAL_PATH)
weather_df = weather_df.astype(np.float16)
weather_df['time_intervall_length'] = weather_df.time_intervall_length.astype(np.uint8)
weather_df.head(2)

,min_temperature,max_temperature,mean_temperature,mean_mean_wind_speed,mean_total_cloud_cover,sum_precipitation,time_intervall_length
MESS_DATUM,,,,,,,
2019-01-01 00:00:00,7.398438,7.398438,7.398438,2.800781,8.0,0.0,1
2019-01-01 01:00:00,7.699219,7.699219,7.699219,2.900391,8.0,0.0,1


In [9]:
weather_df = weather_df.drop(columns=['mean_temperature'])
trips_aggregated_df.join(weather_df, on='datetime_start_floored')
trips_aggregated_df = pd.merge(trips_aggregated_df, weather_df,  how='left', 
                    left_on=['datetime_start_floored','time_interval_length'],
                    right_on = ['MESS_DATUM','time_intervall_length'])

In [10]:
del weather_df
gc.collect()

0

In [11]:
trips_aggregated_df = trips_aggregated_df.drop(columns=['datetime_start_floored'])

In [12]:
hexagons_with_pois_df = pd.read_parquet(HEXAGON_WITH_POIS_PATH)
hexagons_with_pois_df = hexagons_with_pois_df.drop(columns=["hex_and_neighbors"])
hexagons_with_pois_df = hexagons_with_pois_df.astype(
    {
        "h3_res": np.uint16,
        "sustenance_poi": np.uint16,
        "public_transport_poi": np.uint16,
        "education_poi": np.uint16,
        "arts_and_culture_poi": np.uint16,
        "sports_poi": np.uint16,
    }
)

In [13]:
trips_aggregated_df = pd.merge(
    trips_aggregated_df, hexagons_with_pois_df, left_on="start_hex_id", right_on="hex"
)
trips_aggregated_df = trips_aggregated_df.drop(columns={"hex", "h3_res_x"})

# add '_start' suffix to poi columns
trips_aggregated_df = trips_aggregated_df.rename(
    columns={
        "sustenance_poi": "sustenance_poi_start",
        "public_transport_poi": "public_transport_poi_start",
        "education_poi": "education_poi_start",
        "arts_and_culture_poi": "arts_and_culture_poi_start",
        "sports_poi": "sports_poi_start",
    }
)
trips_aggregated_df.head(2)

,start_hex_id,end_hex_id,demand,time_interval_length,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,...,mean_mean_wind_speed,mean_total_cloud_cover,sum_precipitation,time_intervall_length,h3_res_y,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start
0,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,0.799805,0.0,0.0,1,7,862,503,101,40,45
1,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,0.799805,0.0,0.0,1,7,862,503,101,40,45


In [14]:
trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_pois_df, left_on="end_hex_id", right_on="hex")
trips_aggregated_df = trips_aggregated_df.drop(columns={"hex", "h3_res_y"})

# add '_end' suffix to poi columns
trips_aggregated_df = trips_aggregated_df.rename(
    columns={
        "sustenance_poi": "sustenance_poi_end",
        "public_transport_poi": "public_transport_poi_end",
        "education_poi": "education_poi_end",
        "arts_and_culture_poi": "arts_and_culture_poi_end",
        "sports_poi": "sports_poi_end",
    }
)
trips_aggregated_df.head(2)

,start_hex_id,end_hex_id,demand,time_interval_length,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,...,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start,h3_res,sustenance_poi_end,public_transport_poi_end,education_poi_end,arts_and_culture_poi_end,sports_poi_end
0,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,503,101,40,45,7,862,503,101,40,45
1,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,503,101,40,45,7,862,503,101,40,45


In [15]:
del hexagons_with_pois_df
gc.collect()

0

In [16]:
hexagons_with_land_use_df = pd.read_parquet(HEXAGONS_WITH_LAND_USE_PATH)
hexagons_with_land_use_df = hexagons_with_land_use_df.astype(np.float16)
hexagons_with_land_use_df.head(2)

land_use,land_use_1,land_use_2,land_use_3,land_use_4,land_use_5,land_use_6,land_use_7,land_use_8,land_use_9,land_use_10,...,land_use_13,land_use_14,land_use_15,land_use_16,land_use_17,land_use_18,land_use_19,land_use_20,land_use_21,land_use_22
hexagon_id,,,,,,,,,,,,,,,,,,,,,
871f1a140ffffff,0.000000,0.000000,0.000000,0.000000,0.006058,0.000000,0.019272,0.015480,0.000000,0.319092,...,0.000000,0.0,0.0,0.007874,0.0,0.000000,0.0,0.001923,0.008865,0.0
871f1a144ffffff,0.148926,0.155518,0.016571,0.084351,0.135986,0.058533,0.071045,0.082703,0.004414,0.031219,...,0.000406,0.0,0.0,0.000000,0.0,0.004196,0.0,0.070251,0.000000,0.0


In [17]:
trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_land_use_df.add_prefix("start_"), left_on="start_hex_id", right_on="hexagon_id")
trips_aggregated_df.head(2)

,start_hex_id,end_hex_id,demand,time_interval_length,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,...,start_land_use_13,start_land_use_14,start_land_use_15,start_land_use_16,start_land_use_17,start_land_use_18,start_land_use_19,start_land_use_20,start_land_use_21,start_land_use_22
0,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
1,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0


In [18]:
trips_aggregated_df = pd.merge(trips_aggregated_df, hexagons_with_land_use_df.add_prefix("end_"), left_on="end_hex_id", right_on="hexagon_id")
trips_aggregated_df.head(2)

,start_hex_id,end_hex_id,demand,time_interval_length,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,...,end_land_use_13,end_land_use_14,end_land_use_15,end_land_use_16,end_land_use_17,end_land_use_18,end_land_use_19,end_land_use_20,end_land_use_21,end_land_use_22
0,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
1,871f1a164ffffff,871f1a164ffffff,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0


In [19]:
del hexagons_with_land_use_df
gc.collect()

0

In [20]:
trips_aggregated_df.memory_usage(index=True, deep=True).sum() / 1024**2

16080.999231338501

In [21]:
start_hex_id_map = dict(enumerate(pd.Categorical(trips_aggregated_df.start_hex_id.unique()).categories))
end_hex_id_map = dict(enumerate(pd.Categorical(trips_aggregated_df.end_hex_id.unique()).categories))
start_hex_id_map = {v: k for k, v in start_hex_id_map.items()}
end_hex_id_map = {v: k for k, v in end_hex_id_map.items()}

In [22]:
print(max(start_hex_id_map.values()))
print(max(end_hex_id_map.values()))

956
969


In [23]:
trips_aggregated_df["start_hex_id"] = trips_aggregated_df["start_hex_id"].map(start_hex_id_map).astype(np.uint16)
trips_aggregated_df["end_hex_id"] = trips_aggregated_df["end_hex_id"].map(end_hex_id_map).astype(np.uint16)

In [24]:
trips_aggregated_df.to_feather(MODEL_DATA_PATH)

In [25]:
import os

In [26]:
trips_aggregated_df

,start_hex_id,end_hex_id,demand,time_interval_length,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,...,end_land_use_13,end_land_use_14,end_land_use_15,end_land_use_16,end_land_use_17,end_land_use_18,end_land_use_19,end_land_use_20,end_land_use_21,end_land_use_22
0,6,6,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
1,6,6,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
2,6,6,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
3,6,6,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
4,6,6,1,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100647,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51883533,472,780,1,6,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090820,0.0,0.0
51883534,472,780,1,6,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090820,0.0,0.0
51883535,472,780,1,24,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090820,0.0,0.0
51883536,472,780,1,24,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090820,0.0,0.0


In [27]:
if not os.path.exists(MODEL_DATA_DIR_PATH):
	os.makedirs(MODEL_DATA_DIR_PATH)
for h3_res in CALC_H3_RESOLUTIONS:
    for t in CALC_TIME_INTERVAL_LENGTHS:
        trips_aggregated_df[
            (trips_aggregated_df.h3_res == h3_res)
            & (trips_aggregated_df.time_interval_length == t)
        ].reset_index(drop=True).to_feather(
            os.path.join(MODEL_DATA_DIR_PATH, f"{h3_res}_{t}.feather")
        )

In [28]:
del trips_aggregated_df
gc.collect()

0

In [29]:
for h3_res in CALC_H3_RESOLUTIONS:
    for t in CALC_TIME_INTERVAL_LENGTHS:
        model_data = pd.read_feather(os.path.join(MODEL_DATA_DIR_PATH, f"{h3_res}_{t}.feather"))

        start_hex_dummies = pd.get_dummies(model_data.start_hex_id, prefix="start_")
        end_hex_dummies = pd.get_dummies(model_data.end_hex_id, prefix="end_")
        model_data = pd.concat([model_data, start_hex_dummies, end_hex_dummies], axis=1)
        model_data = model_data.drop(columns=['start_hex_id', 'end_hex_id'])

        model_data.to_feather(os.path.join(MODEL_DATA_DIR_PATH, f"{h3_res}_{t}.feather"))